In [ ]:
!pip install pydub

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as pt

from scipy.io import wavfile
import librosa
import librosa.display
import pydub
from scipy.io import wavfile

import os
import time
import zipfile
import io
import sys


In [ ]:
TEST_FILE_PATH = "/content/drive/MyDrive/CS529/data/test"

In [ ]:

def dir_to_df():
    #if train: # training data

    # Directory path
    directory = './'

    # Initialize an empty list to store file names
    au_files = []

    # Iterate through the files in the directory
    for filename in os.listdir(directory):
        # Check if the file has the ".au" extension
        if filename.endswith(".au"):
            # Append the file name to the list
            au_files.append(filename)

    # Print the list of .au files
    print(au_files)
    return au_files



# Path to the audio file
#for i in range(101):  # Iterate from 0 to 100
    # Format the number with leading zeros to ensure it has 5 digits
 #   padded_number = str(i).zfill(5)
  #  audio_file_suffix = padded_number + ".au"
    #audio_file = 'resources/train/blues/blues.00000.au'


def get_interquartile_range(data):
  """
  data : numpy 2D matrix
  """

  # get upper quartile
  Q3 = np.percentile( data , 75, axis=1)

  # get lower quartile
  Q1 = np.percentile(data, 25, axis=1)

  IQR =  Q3 - Q1
  return IQR

def get_percentiles(data, percentiles):
  """

  data : 2-D numpy array
  percentiles : list w/ percentile values

  Returns:
    percentile across each row in the 2-D matrix

  """
  results = []

  for p_ in percentiles:

    results.append( np.percentile(data, p_, axis=1 ) )


  return tuple(results)

def process_au(au_path):
    # Load the audio file using librosa
    audio_file = au_path
    audio_data, sample_rate = librosa.load(audio_file, sr=None)

    # Extract features using librosa
    # Example: compute the Mel-Frequency Cepstral Coefficients (MFCCs)
    mfccs = librosa.feature.mfcc(y=audio_data, sr=sample_rate, n_mfcc=13)
    stft = np.abs(librosa.stft(y=audio_data))
    chroma = librosa.feature.chroma_stft(y=audio_data, sr=sample_rate)
    contrast = librosa.feature.spectral_contrast(y=audio_data, sr=sample_rate)
    zcr = librosa.feature.zero_crossing_rate(y=audio_data)

    # Calculate the mean across each row
    mean_mfcc = np.mean(mfccs, axis=1)
    mean_stft = np.mean(stft, axis=1)
    mean_chroma = np.mean(chroma, axis=1)
    mean_contrast = np.mean(contrast, axis=1)
    mean_zcr = np.mean(zcr, axis=1)


    # get minimum and maximum across each row
    min_mfcc = np.min(mfccs, axis=1)
    min_stft = np.min(stft, axis=1)
    min_chroma = np.min(chroma, axis=1)
    min_contrast = np.min(contrast, axis=1)
    min_zcr = np.min(zcr, axis=1)

    max_mfcc = np.max(mfccs, axis=1)
    max_stft = np.max(stft, axis=1)
    max_chroma = np.max(chroma, axis=1)
    max_contrast = np.max(contrast, axis=1)
    max_zcr = np.max(zcr, axis=1)


    # get median across each row
    median_mfcc = np.median(mfccs, axis=1)
    median_stft = np.median(stft, axis=1)
    median_chroma = np.median(chroma, axis=1)
    median_contrast = np.median(contrast, axis=1)
    median_zcr = np.median(zcr, axis=1)


    # get interquartile range across each row
    iq_mfcc = np.percentile( mfccs , 75, axis=1) - np.percentile( mfccs , 25, axis=1)
    iq_stft = np.percentile( stft , 75, axis=1) - np.percentile( stft , 25, axis=1)
    iq_chroma = np.percentile( chroma , 75, axis=1) - np.percentile( chroma , 25, axis=1)
    iq_contrast = np.percentile( contrast , 75, axis=1) - np.percentile( contrast , 25, axis=1)
    iq_zcr = np.percentile( zcr , 75, axis=1) - np.percentile( zcr , 25, axis=1)



    """
    NOTE: note taking the percentiles of stft.
    """


    # get percentiles across each row
    percentiles = [12.5, 25.0, 37.5, 62.5, 75.0, 87.5]

    p_12_5_mfcc , p_25_mfcc, p_37_5_mfcc, p_62_5_mfcc , p_75_mfcc , p_87_5_mfcc = get_percentiles(mfccs, percentiles)
    p_12_5_chroma , p_25_chroma, p_37_5_chroma, p_62_5_chroma , p_75_chroma , p_87_5_chroma = get_percentiles(chroma, percentiles)
    p_12_5_constrast , p_25_contrast, p_37_5_contrast, p_62_5_contrast , p_75_contrast , p_87_5_contrast = get_percentiles(contrast, percentiles)
    p_12_5_zcr , p_25_zcr, p_37_5_zcr, p_62_5_zcr , p_75_zcr , p_87_5_zcr = get_percentiles(zcr, percentiles)

    #print('p_12_5_mfcc shape :', p_12_5_mfcc.shape)
    #print(p_12_5_mfcc.shape, p_25_mfcc.shape, p_37_5_mfcc.shape, p_62_5_mfcc.shape , p_75_mfcc.shape , p_87_5_mfcc.shape)

    # Reshape to a column vector
    #mean_mfcc = mean_mfcc.reshape(-1, 1)
    #mean_stft = mean_stft.reshape(-1, 1)
    #mean_chroma = mean_chroma.reshape(-1, 1)
    #mean_contrast = mean_contrast.reshape(-1, 1)
    #mean_zcr = mean_zcr.reshape(-1, 1)

    # Concatenate the vectors
    concatenated_vectors = np.concatenate((mean_mfcc, mean_stft,
                                           mean_chroma, mean_contrast, mean_zcr,

                                           min_mfcc, #min_stft,
                                           min_chroma, min_contrast, min_zcr,

                                           max_mfcc, #max_stft,
                                           max_chroma, max_contrast, max_zcr,

                                           median_mfcc, #median_stft,
                                           median_chroma, median_contrast, median_zcr,

                                           iq_mfcc, #iq_stft,
                                           iq_chroma, iq_contrast, iq_zcr,

                                           p_12_5_mfcc , p_25_mfcc, p_37_5_mfcc, p_62_5_mfcc , p_75_mfcc , p_87_5_mfcc,
                                           p_12_5_chroma , p_25_chroma, p_37_5_chroma, p_62_5_chroma , p_75_chroma , p_87_5_chroma,
                                           p_12_5_constrast , p_25_contrast, p_37_5_contrast, p_62_5_contrast , p_75_contrast , p_87_5_contrast,
                                           p_12_5_zcr , p_25_zcr, p_37_5_zcr, p_62_5_zcr , p_75_zcr , p_87_5_zcr

                                           ), axis=0)
    print("New_vector Shape:", concatenated_vectors.shape)


    return concatenated_vectors #.reshape(-1, 1)

    #print("hello")


In [ ]:
"""
This function only supports the logic behind creating train data w/ label

"""

def create_test_dataset(path):

  df_list = []

  # check if a path exist
  #for label in labels :  # there are no labels in the test directory

    #print(label)
  data = []

  # create new path
  #label_path = os.path.join(path, label)
  #print(label_path)

  # after the path is created, read each .au files in that PATH
  for filename in os.listdir(path):
    if filename.endswith('.au'):

      # Construct the full path to the audio file
      audio_file_path = os.path.join(path, filename)
      #print(audio_file_path)

      # apply the process_au
      data.append( process_au(audio_file_path) )

  data = pd.DataFrame( np.array(data) )




  #df_list.append(data)


  # At this point, df is list of df with of shape (90, 1389) for train data.
  # I want them to merge into a single dataframe with shape (90*length_of_df. 1389)
  #df =  pd.concat(data, ignore_index=True)

  return data




In [ ]:
start_time = time.time()
test_df = create_test_dataset(TEST_FILE_PATH)
end_time = time.time()

New_vector Shape: (1388,)
New_vector Shape: (1388,)
New_vector Shape: (1388,)
New_vector Shape: (1388,)
New_vector Shape: (1388,)
New_vector Shape: (1388,)
New_vector Shape: (1388,)
New_vector Shape: (1388,)
New_vector Shape: (1388,)
New_vector Shape: (1388,)
New_vector Shape: (1388,)
New_vector Shape: (1388,)
New_vector Shape: (1388,)
New_vector Shape: (1388,)
New_vector Shape: (1388,)
New_vector Shape: (1388,)
New_vector Shape: (1388,)
New_vector Shape: (1388,)
New_vector Shape: (1388,)
New_vector Shape: (1388,)
New_vector Shape: (1388,)
New_vector Shape: (1388,)
New_vector Shape: (1388,)
New_vector Shape: (1388,)
New_vector Shape: (1388,)
New_vector Shape: (1388,)
New_vector Shape: (1388,)
New_vector Shape: (1388,)
New_vector Shape: (1388,)
New_vector Shape: (1388,)
New_vector Shape: (1388,)
New_vector Shape: (1388,)
New_vector Shape: (1388,)
New_vector Shape: (1388,)
New_vector Shape: (1388,)
New_vector Shape: (1388,)
New_vector Shape: (1388,)
New_vector Shape: (1388,)
New_vector S

In [ ]:
test_df.shape

(100, 1388)

In [ ]:
FOLDER_PATH = ( os.path.join( os.getcwd(), 'drive', 'MyDrive', 'CS529' ))

In [ ]:
test_df.to_csv(os.path.join(FOLDER_PATH, 'test.csv'), index=False)